# Direct Solvers

The purpose of this notebook is to compare commonly available direct solvers from python interface. Here we focus on multi-threaded solvers that can be run on a single node with several cores.
For now, we compare SuperLU, UMFPACK and Intel MKL PARDISO. SuperLU is the default solver in scipy, UMFPACK requires the installation of scikit-umfpack and Intel MKL PARDISO requires PyPardiso.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as spla

import ibmos as ib

In [3]:
import scipy.linalg as la

# Case initialization
... from [Unsteady flow around cylinder (Re=200)](../1-Basic/CylinderRe200.ipynb)

In [4]:
s1 = ib.stretching(256, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
s2 = ib.stretching(128, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
x = np.r_[-s2[::-1], s1[1:]]

s = ib.stretching(192, 0.033, 0.20, int(0.5/0.033+16), 16, 16, 0.04)
y = np.r_[-s[::-1], s[1:]]

solver = ib.Solver(x, y, iRe=1/200, Co=0.015/0.033)
del x, y, s1, s2

solver.set_solids(ib.shapes.cylinder("cylinder", 0, 0, 0.5, solver.dxmin))

uBC, vBC = solver.zero_boundary_conditions()
for k in range(4):
    uBC[k][:] = 1

sBC = ((np.zeros(solver.solids[0].l), np.zeros(solver.solids[0].l)), )

u, v, p, *f = solver.reshape(*solver.unpack(solver.zero()))
c, r, ϵ = -10, 1.0, 0.01
u[:,:] = 1.0-ϵ*solver.fluid.u.y[:,np.newaxis]*np.exp(-((solver.fluid.u.x[np.newaxis,:]-c)**2 + solver.fluid.u.y[:,np.newaxis]**2)/r**2)
v[:,:] = ϵ*(solver.fluid.v.x[np.newaxis,:]-c)*np.exp(-((solver.fluid.v.x[np.newaxis,:]-c)**2 + solver.fluid.v.y[:,np.newaxis]**2)/r**2)
x0 = solver.pack(u, v, p, f)

In [5]:
n = 10

# SuperLU

In [6]:
solver.set_solver(ib.tools.solver_superlu)

## Without fractional step method

In [7]:
solver.set_fractional_step(False)

In [8]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  1.39370e+03  8.94021e+04  2.32815e+02 -5.06399e-08  5.70587e-16 
CPU times: user 9min 9s, sys: 12.7 s, total: 9min 22s
Wall time: 28.9 s


In [9]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  3.94895e+02  8.88363e+04  2.11691e+00  4.16394e-08  5.48658e-16 
       2  3.00000e-02  3.90576e+02  1.25370e+03  1.98338e+00  4.32569e-08  5.53124e-16 
       3  4.50000e-02  3.87237e+02  1.20493e+03  1.84122e+00  4.44633e-08  5.45656e-16 
       4  6.00000e-02  3.85533e+02  7.66895e+02  1.72072e+00  4.56213e-08  5.46340e-16 
       5  7.50000e-02  3.84987e+02  2.81708e+02  1.62136e+00  4.67405e-08  5.48010e-16 
       6  9.00000e-02  3.84852e+02  8.42332e+01  1.54062e+00  4.78308e-08  5.47917e-16 
       7  1.05000e-01  3.84604e+02  1.41523e+02  1.47479e+00  4.89013e-08  5.46949e-16 
       8  1.20000e-01  3.84162e+02  2.65593e+02  1.41967e+00  4.99592e-08  5.52095e-16 
       9  1.35000e-01  3.83775e+02  2.56340e+02  1.37208e+00  5.10146e-08  5.50366e-16 
      10  1.50000e-01  3.83613e+02  1.14614e+02  1.33072e+00  5.20720e-08  5.45341e-16 
CPU times: user 1min 27s, sys: 2

## With fractional step method

In [10]:
solver.set_fractional_step(True)

In [11]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -5.06434e-08  2.08546e-16  8.70706e-15 
CPU times: user 9min 16s, sys: 11.2 s, total: 9min 27s
Wall time: 24.6 s


In [12]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88307e+04  2.11836e+00  4.16408e-08  2.15570e-16  1.23345e-14 
       2  3.00000e-02  3.90526e+02  1.24713e+03  1.98458e+00  4.32535e-08  2.14912e-16  1.13860e-14 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84210e+00  4.44603e-08  2.16107e-16  1.05533e-14 
       4  6.00000e-02  3.85515e+02  7.64226e+02  1.72136e+00  4.56186e-08  2.15647e-16  9.95153e-15 
       5  7.50000e-02  3.84973e+02  2.80624e+02  1.62183e+00  4.67380e-08  2.17006e-16  8.96850e-15 
       6  9.00000e-02  3.84840e+02  8.37754e+01  1.54098e+00  4.78288e-08  2.16056e-16  9.01459e-15 
       7  1.05000e-01  3.84593e+02  1.40824e+02  1.47506e+00  4.88990e-08  2.15227e-16  9.94795e-15 
       8  1.20000e-01  3.84154e+02  2.64729e+02  1.41987e+00  4.99570e-08  2.15817e-16  8.80619e-15 
       9  1.35000e-01  3.83768e+02  2.55747e+02  1.37224e+00  5.10123e-08  2.15583e-16  8.6

# UMFPACK

In [13]:
solver.set_solver(ib.tools.solver_umfpack)

## Without fracional step method

In [14]:
solver.set_fractional_step(False)

In [15]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  1.39370e+03  8.94021e+04  2.32815e+02 -5.06410e-08  1.49865e-16 
CPU times: user 2min 34s, sys: 4.24 s, total: 2min 38s
Wall time: 6.29 s


In [16]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  3.94895e+02  8.88363e+04  2.11691e+00  4.16401e-08  1.48094e-16 
       2  3.00000e-02  3.90576e+02  1.25370e+03  1.98338e+00  4.32570e-08  1.47104e-16 
       3  4.50000e-02  3.87237e+02  1.20493e+03  1.84122e+00  4.44633e-08  1.47948e-16 
       4  6.00000e-02  3.85533e+02  7.66895e+02  1.72072e+00  4.56212e-08  1.47504e-16 
       5  7.50000e-02  3.84987e+02  2.81708e+02  1.62136e+00  4.67404e-08  1.46890e-16 
       6  9.00000e-02  3.84852e+02  8.42332e+01  1.54062e+00  4.78311e-08  1.47322e-16 
       7  1.05000e-01  3.84604e+02  1.41523e+02  1.47479e+00  4.89013e-08  1.47237e-16 
       8  1.20000e-01  3.84162e+02  2.65593e+02  1.41967e+00  4.99591e-08  1.47214e-16 
       9  1.35000e-01  3.83775e+02  2.56340e+02  1.37208e+00  5.10144e-08  1.47350e-16 
      10  1.50000e-01  3.83613e+02  1.14614e+02  1.33072e+00  5.20722e-08  1.47834e-16 
CPU times: user 1min 7s, sys: 1.

## With fractional step method

In [17]:
solver.set_fractional_step(True)

In [18]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -5.06386e-08  1.07039e-16  2.75241e-15 
CPU times: user 2min 1s, sys: 3.26 s, total: 2min 5s
Wall time: 5.26 s


In [19]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88307e+04  2.11836e+00  4.16360e-08  1.30699e-16  4.40659e-15 
       2  3.00000e-02  3.90526e+02  1.24713e+03  1.98458e+00  4.32535e-08  1.30985e-16  3.64973e-15 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84210e+00  4.44603e-08  1.30428e-16  3.54322e-15 
       4  6.00000e-02  3.85515e+02  7.64226e+02  1.72136e+00  4.56186e-08  1.30927e-16  2.95364e-15 
       5  7.50000e-02  3.84973e+02  2.80624e+02  1.62183e+00  4.67380e-08  1.30452e-16  3.11873e-15 
       6  9.00000e-02  3.84840e+02  8.37754e+01  1.54098e+00  4.78288e-08  1.30966e-16  3.26252e-15 
       7  1.05000e-01  3.84593e+02  1.40824e+02  1.47506e+00  4.88990e-08  1.29680e-16  3.02610e-15 
       8  1.20000e-01  3.84154e+02  2.64729e+02  1.41987e+00  4.99570e-08  1.30773e-16  3.18780e-15 
       9  1.35000e-01  3.83768e+02  2.55747e+02  1.37224e+00  5.10123e-08  1.30223e-16  2.7

# PARDISO

In [20]:
solver.set_solver(ib.tools.solver_pardiso)

## Without fractional step method

In [21]:
solver.set_fractional_step(False)

In [22]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  1.39370e+03  8.94021e+04  2.32815e+02 -5.03008e-08  4.66079e-16 
CPU times: user 23 s, sys: 1.49 s, total: 24.5 s
Wall time: 2.04 s


In [23]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) 
       1  1.50000e-02  3.94895e+02  8.88363e+04  2.11691e+00  4.13148e-08  1.63358e-16 
       2  3.00000e-02  3.90576e+02  1.25370e+03  1.98338e+00  4.32551e-08  1.63082e-16 
       3  4.50000e-02  3.87237e+02  1.20493e+03  1.84122e+00  4.44612e-08  1.63185e-16 
       4  6.00000e-02  3.85533e+02  7.66895e+02  1.72072e+00  4.56198e-08  1.62270e-16 
       5  7.50000e-02  3.84987e+02  2.81708e+02  1.62136e+00  4.67397e-08  1.62798e-16 
       6  9.00000e-02  3.84852e+02  8.42332e+01  1.54062e+00  4.78309e-08  1.63111e-16 
       7  1.05000e-01  3.84604e+02  1.41523e+02  1.47479e+00  4.89009e-08  1.62135e-16 
       8  1.20000e-01  3.84162e+02  2.65593e+02  1.41967e+00  4.99587e-08  1.62210e-16 
       9  1.35000e-01  3.83775e+02  2.56340e+02  1.37208e+00  5.10140e-08  1.62593e-16 
      10  1.50000e-01  3.83613e+02  1.14614e+02  1.33072e+00  5.20719e-08  1.62415e-16 
CPU times: user 31.7 s, sys: 1.2

## With fractional step method

In [24]:
solver.set_fractional_step(True)

In [25]:
%time x1, *_ = solver.steps(x0, uBC, vBC, sBC, number=1, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  1.39365e+03  8.93986e+04  2.32804e+02 -5.06391e-08  1.78456e-16  4.65474e-15 
CPU times: user 43.3 s, sys: 2.83 s, total: 46.2 s
Wall time: 2.64 s


In [26]:
%time x, *_ = solver.steps(x1, uBC, vBC, sBC, number=n, checkSolvers=True)

       k            t          x_2       dxdt_2  cylinder_fx  cylinder_fy rel.error(A) rel.error(C) 
       1  1.50000e-02  3.94809e+02  8.88307e+04  2.11836e+00  4.16364e-08  1.96752e-16  6.67270e-15 
       2  3.00000e-02  3.90526e+02  1.24713e+03  1.98458e+00  4.32535e-08  1.96210e-16  6.18182e-15 
       3  4.50000e-02  3.87209e+02  1.20003e+03  1.84210e+00  4.44603e-08  1.95143e-16  6.08125e-15 
       4  6.00000e-02  3.85515e+02  7.64226e+02  1.72136e+00  4.56186e-08  1.96063e-16  5.13808e-15 
       5  7.50000e-02  3.84973e+02  2.80624e+02  1.62183e+00  4.67380e-08  1.96144e-16  4.59882e-15 
       6  9.00000e-02  3.84840e+02  8.37754e+01  1.54098e+00  4.78288e-08  1.94302e-16  5.07076e-15 
       7  1.05000e-01  3.84593e+02  1.40824e+02  1.47506e+00  4.88990e-08  1.95066e-16  5.05513e-15 
       8  1.20000e-01  3.84154e+02  2.64729e+02  1.41987e+00  4.99570e-08  1.95173e-16  4.65776e-15 
       9  1.35000e-01  3.83768e+02  2.55747e+02  1.37224e+00  5.10123e-08  1.94311e-16  4.5